In [1]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
#tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
#session_conf = tf.ConfigProto()
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.applications.vgg16 import VGG16
from keras.layers import LSTM
#import numpy as np
import glob
from imageio import imread
from skimage.transform import resize as imresize
from keras.layers import LeakyReLU
from keras.applications.inception_v3 import preprocess_input

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 120

In [3]:
def bring_data_from_directory():
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

    train_generator = datagen.flow_from_directory(
          '/part4/fps15Videos/data/train',
          target_size=(224, 224),
          batch_size=batch_size,
          class_mode='categorical',
          shuffle=True,
          classes=['diaper_change','feeding','patting'])

    validation_generator = datagen.flow_from_directory(
          '/part4/fps15Videos/data/validate',
          target_size=(224, 224),
         batch_size=batch_size,
          class_mode='categorical',  
          shuffle=True,
          classes=['diaper_change','feeding','patting'])

    return train_generator,validation_generator

In [4]:
train_generator,validation_generator = bring_data_from_directory()

Found 35679 images belonging to 3 classes.
Found 7174 images belonging to 3 classes.


In [5]:
pretrained_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='avg')
pretrained_model.trainable = False

In [6]:
model = Sequential()
model.add(pretrained_model)
model.add(Dense(2048))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es =   EarlyStopping(monitor='val_loss', patience=8, verbose=0)

In [7]:
transfer = model.fit_generator(train_generator,epochs=40,validation_data=validation_generator,callbacks=[es])


Epoch 1/40
298/298 [==============================] - 639s 2s/step - loss: 0.9444 - accuracy: 0.8043 - val_loss: 2.5287 - val_accuracy: 0.6287
Epoch 2/40
298/298 [==============================] - 633s 2s/step - loss: 0.2630 - accuracy: 0.9008 - val_loss: 4.5252 - val_accuracy: 0.6156
Epoch 3/40
298/298 [==============================] - 629s 2s/step - loss: 0.2034 - accuracy: 0.9266 - val_loss: 3.4329 - val_accuracy: 0.6239
Epoch 4/40
298/298 [==============================] - 629s 2s/step - loss: 0.1749 - accuracy: 0.9360 - val_loss: 3.1399 - val_accuracy: 0.6110
Epoch 5/40
298/298 [==============================] - 627s 2s/step - loss: 0.1875 - accuracy: 0.9321 - val_loss: 4.1678 - val_accuracy: 0.5998
Epoch 6/40
298/298 [==============================] - 627s 2s/step - loss: 0.1620 - accuracy: 0.9400 - val_loss: 3.9057 - val_accuracy: 0.6165
Epoch 7/40
298/298 [==============================] - 627s 2s/step - loss: 0.1483 - accuracy: 0.9466 - val_loss: 4.2591 - val_accuracy: 0.575

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 6147      
Total params: 26,005,283
Trainable params: 4,202,499
Non-trainable params: 21,802,784
_________________________________________________________________


In [9]:
I_layer = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [10]:
I_layer.save('Tuned_inception_V3_30april_15fps_15patients.h5')